# Variables

In [ ]:
YOUR_NAME = 'chris'

AWS_PROFILE = 'CitiesUserPermissionSet'


# List of cities to process
cities = ["Belo Horizonte", "Campinas", "Bogota", "Nairobi", "Bamako", 
        "Lagos", "Accra", "Abidjan", "Mogadishu", "Cape Town", 
        "Maputo", "Luanda"]

test_cities = ["Belo Horizonte"]
#cities = test_cities

cities = [city.replace(' ', '_') for city in cities]

number_of_cities = len(cities)

print(f'City count: {number_of_cities}')

grid_size = 200

# Setup

In [ ]:
%load_ext autoreload

In [ ]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3

session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE


s3.list_buckets()

In [ ]:
import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=4,
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")


# RUN

In [ ]:
from dask import compute

%autoreload
from metrics_groupby import metrics

print(f'Processing {len(cities)} cities: {cities}')

# Create delayed tasks for counting
grid_calculations = []

for city_name in cities:
    grid_calc = metrics(city_name, "chris", grid_size=grid_size)
    grid_calculations.append(grid_calc)

#visualize(*grid_calulations)
calculated_grids = compute(*grid_calculations)
calculated_grids




In [ ]:
# Sum the total number of grid cells
total_grid_cells = sum([grid_cells for grid_cells, path in calculated_grids])
print(f'Total grid cells: {total_grid_cells}')